In [2]:

"""
Training Workflow:
1. Run your Docker Freqtrade Jupyterlab
2. Download newest data (TODO: Kaggle lib script to remove old data then upload new one, cuts step 3 through 4)
3. Zip downloaded data
4. Upload to Kaggle
5. Open Kaggle notebook that cointains training ML model (or create a new one)
6. Train model
7. Predict then do some EDA
8. Backtest using that model

Backtest Workflow:
1. Create new preset and IStrategy
2. Tweak the strategy parameters (E.G: Good point thresholding, Adding TA filters like buy on divergence only, Hyperopting)
3. Run freqtrade backtesting in notebook. Timerange: After val period when training
4. Are the equity curve good enough?
   Yes: Save to "candidate" then Dry run for 1 week
   No: Return to step 2
5. Are the dry run good enough?
   Yes: Live run!
   No: Back to step 2 or training
"""
def resolve_imports():
    import os
    import stackprinter
    stackprinter.set_excepthook(style='darkbg2')  # for jupyter notebooks try style='lightbg'


    if "freqtrade" not in os.listdir():
        os.chdir("..")
        import nest_asyncio
        nest_asyncio.apply()

resolve_imports()

In [3]:
resolve_imports()

from freqtrade.nbtools.preset import Preset


def strategy_func():
    """ Start Strategy Coding """
    from typing import Dict, List, Tuple, Union, Optional, Callable
    from pandas import DataFrame
    import numpy as np  # noqa
    import pandas as pd  # noqa
    from freqtrade.nbtools.strategy import INbStrategy
    # --------------------------------
    # Add your lib to import here
    import talib.abstract as ta
    import freqtrade.vendor.qtpylib.indicators as qtpylib
    import sklearn
    import pickle
    import marshal
    import types

    # Load ML models so it only loaded once (Cached)
    def load_model():
        pass

    # Start Configure Strategy
    class NotebookStrategy(INbStrategy):
        timeframe: str = "15m"
        minimal_roi: Dict[int, float] = {"0": 0.02, "30": 0.01}
        stoploss: float = -0.01
        startup_candle_count: int = 100
        
        def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            dataframe["ema_10"] = ta.EMA(dataframe["close"], timeperiod=10)
            dataframe["ema_20"] = ta.EMA(dataframe["close"], timeperiod=20)
            return dataframe

        def populate_buy_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            dataframe.loc[
                (
                    (dataframe['ema_10'] > dataframe['ema_20']) &
                    (dataframe['volume'] > 0)  # Make sure Volume is not 0
                ),
                'buy'] = 1
            return dataframe

        def populate_sell_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
            dataframe.loc[
                (
                    (dataframe['ema_10'] < dataframe['ema_20']) &
                    (dataframe['volume'] > 0)  # Make sure Volume is not 0
                ),
                'sell'] = 1
            return dataframe
    
    """End Strategy Coding"""


# Start Configure Backtesting
preset = Preset(
    name = "ma_crossover",
    datadir = "../mount/data/binance",
    exchange = "binance",
    timerange = "20210501-20210601",
    stake_amount = 15,
    starting_balance = 1000,
    max_open_trades = 100,
    fee = 0.001,
    pairs = ["BTC/USDT"],
)
stats, summary = preset.backtest_by_strategy_func(strategy_func)
print(summary)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jliberooo (use `wandb login --relogin` to force relogin)


wandb: Adding directory to artifact (.\.temp\ma_crossover__backtest-2021-07-12_22-55-41)... Done. 0.0s


Result for strategy NotebookStrategy
========================================================== BACKTESTING REPORT =========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
| BTC/USDT |    329 |          -0.28 |         -92.92 |           -13.952 |          -1.40 |        0:51:00 |   117     0   212  35.6 |
|    TOTAL |    329 |          -0.28 |         -92.92 |           -13.952 |          -1.40 |        0:51:00 |   117     0   212  35.6 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+----

In [3]:
import pandas as pd
from copy import deepcopy

all_stats = deepcopy(stats)
trades = pd.DataFrame(deepcopy(stats["strategy"]["NotebookStrategy"]["trades"]))
trades_summary = deepcopy(stats["strategy"]["NotebookStrategy"])

del trades_summary["trades"]
del trades_summary["locks"]
del trades_summary["best_pair"]
del trades_summary["worst_pair"]
del trades_summary["results_per_pair"]
del trades_summary["sell_reason_summary"]
del trades_summary["left_open_trades"]

metadata = {}
metadata["preset_name"] = "self.name"
metadata["backtest_date"] = "get_readable_date()"
metadata["leverage"] = 1
metadata["direction"] = "long"
metadata["is_hedging"] = False
metadata["num_pairs"] = len(trades_summary["pairlist"])
metadata["data_source"] = "self.config['exchange']"
metadata["win_rate"] = trades_summary["wins"] / trades_summary["total_trades"]
metadata["avg_profit_winners_abs"] = trades.loc[trades["profit_abs"] >= 0, "profit_abs"].dropna().mean()
metadata["avg_profit_losers_abs"] = trades.loc[trades["profit_abs"] < 0, "profit_abs"].dropna().mean()
metadata["sum_profit_winners_abs"] = trades.loc[trades["profit_abs"] >= 0, "profit_abs"].dropna().sum()
metadata["sum_profit_losers_abs"] = trades.loc[trades["profit_abs"] < 0, "profit_abs"].dropna().sum()
metadata["profit_factor"] = metadata["sum_profit_winners_abs"] / abs(metadata["sum_profit_losers_abs"])
metadata["profit_per_drawdown"] = trades_summary["profit_total_abs"] / abs(trades_summary["max_drawdown_abs"])
metadata["expectancy_abs"] = (
    (metadata["win_rate"] * metadata["avg_profit_winners_abs"]) + 
    ((1 - metadata["win_rate"]) * metadata["avg_profit_losers_abs"])
)

for key, value in trades_summary.items():
    is_valid = any(
        [isinstance(value, it) for it in (str, int, float, bool)] + [value is None],
    )
    if not is_valid:
        trades_summary[key] = str(value)

metadata = {**metadata, **trades_summary}
metadata

{'preset_name': 'self.name',
 'backtest_date': 'get_readable_date()',
 'leverage': 1,
 'direction': 'long',
 'is_hedging': False,
 'num_pairs': 1,
 'data_source': "self.config['exchange']",
 'win_rate': 0.3556231003039514,
 'avg_profit_winners_abs': 0.1759729905128205,
 'avg_profit_losers_abs': -0.16293023740566037,
 'sum_profit_winners_abs': 20.58883989,
 'sum_profit_losers_abs': -34.54121033,
 'profit_factor': 0.5960659656479386,
 'profit_per_drawdown': -0.9504429291487234,
 'expectancy_abs': -0.04240842079027354,
 'total_trades': 329,
 'total_volume': 4935.0,
 'avg_stake_amount': 15.0,
 'profit_mean': -0.002824405957446809,
 'profit_median': -0.01197802,
 'profit_total': -0.013952370439999999,
 'profit_total_abs': -13.95237044,
 'backtest_start': '2021-05-01 00:00:00',
 'backtest_start_ts': 1619827200000,
 'backtest_end': '2021-06-01 00:00:00',
 'backtest_end_ts': 1622505600000,
 'backtest_days': 31,
 'backtest_run_start_ts': 1625751172,
 'backtest_run_end_ts': 1625751172,
 'trades_

In [3]:
metadata = {'preset_name': 'self.name',
 'backtest_date': 'get_readable_date()',
 'leverage': 1,
 'direction': 'long',
 'is_hedging': False,
 'num_pairs': 1,
 'data_source': "self.config['exchange']",
 'win_rate': 0.3556231003039514,
 'avg_profit_winners_abs': 0.1759729905128205,
 'avg_profit_losers_abs': -0.16293023740566037,
 'sum_profit_winners_abs': 20.58883989,
 'sum_profit_losers_abs': -34.54121033,
 'profit_factor': 0.5960659656479386,
 'profit_per_drawdown': -0.9504429291487234,
 'expectancy_abs': -0.04240842079027354,
 'total_trades': 329,
 'total_volume': 4935.0,
 'avg_stake_amount': 15.0,
 'profit_mean': -0.002824405957446809,
 'profit_median': -0.01197802,
 'profit_total': -0.013952370439999999,
 'profit_total_abs': -13.95237044,
 'backtest_start': '2021-05-01 00:00:00',
 'backtest_start_ts': 1619827200000,
 'backtest_end': '2021-06-01 00:00:00',
 'backtest_end_ts': 1622505600000,
 'backtest_days': 31,
 'backtest_run_start_ts': 1625751172,
 'backtest_run_end_ts': 1625751172,
 'trades_per_day': 10.61,
 'market_change': -0.2966096961886866,
 'pairlist': "['BTC/USDT']",
 'stake_amount': 15,
 'stake_currency': 'USDT',
 'stake_currency_decimals': 3,
 'starting_balance': 1000,
 'dry_run_wallet': 1000,
 'final_balance': 986.04762956,
 'rejected_signals': 0,
 'max_open_trades': 1,
 'max_open_trades_setting': 100,
 'timeframe': '15m',
 'timerange': '20210501-20210601',
 'enable_protections': False,
 'strategy_name': 'NotebookStrategy',
 'stoploss': -0.01,
 'trailing_stop': False,
 'trailing_stop_positive': None,
 'trailing_stop_positive_offset': 0.0,
 'trailing_only_offset_is_reached': False,
 'use_custom_stoploss': False,
 'minimal_roi': "{'0': 0.02, '30': 0.01}",
 'use_sell_signal': True,
 'sell_profit_only': False,
 'sell_profit_offset': 0.0,
 'ignore_roi_if_buy_signal': False,
 'backtest_best_day': 0.04493299,
 'backtest_worst_day': -0.16596732000000003,
 'backtest_best_day_abs': 0.67467306,
 'backtest_worst_day_abs': -2.49200548,
 'winning_days': 4,
 'draw_days': 0,
 'losing_days': 28,
 'wins': 117,
 'losses': 212,
 'draws': 0,
 'holding_avg': '0:51:00',
 'holding_avg_s': 3060.0,
 'winner_holding_avg': '0:59:00',
 'winner_holding_avg_s': 3540.0,
 'loser_holding_avg': '0:47:00',
 'loser_holding_avg_s': 2820.0,
 'zero_duration_trades': 0,
 'max_drawdown': 0.9776803600000036,
 'max_drawdown_abs': 14.679861370000008,
 'drawdown_start': '2021-05-01 07:30:00',
 'drawdown_start_ts': 1619854200000.0,
 'drawdown_end': '2021-05-30 00:30:00',
 'drawdown_end_ts': 1622334600000.0,
 'max_drawdown_low': -14.728090550000008,
 'max_drawdown_high': -0.04822918,
 'csum_min': 985.27190945,
 'csum_max': 999.95177082}

from freqtrade.nbtools import remote_utils, constants
import pandas as pd

metadata_df = pd.DataFrame({key:[value] for key, value in metadata.items()})
remote_utils.table_update(metadata_df, constants.PROJECT_NAME, constants.ARTIFACT_TABLE_METADATA, constants.TABLEKEY_METADATA)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jliberooo (use `wandb login --relogin` to force relogin)


Project jliberooo/ft-presets does not contain artifact: "table-metadata:latest"
Creating new table...


In [2]:
from freqtrade.nbtools import remote_utils

remote_utils.cloud_get_presets_df(True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jliberooo (use `wandb login --relogin` to force relogin)


,preset_name,backtest_date,leverage,direction,is_hedging,num_pairs,data_source,win_rate,avg_profit_winners_abs,avg_profit_losers_abs,...,max_drawdown,max_drawdown_abs,drawdown_start,drawdown_start_ts,drawdown_end,drawdown_end_ts,max_drawdown_low,max_drawdown_high,csum_min,csum_max
1,ma_crossover__backtest-2021-07-08_20-45-34,2021-07-08_20-45-34,1,long,False,1,self.config['exchange'],0.355623,0.175973,-0.16293,...,0.97768,14.679861,2021-05-01 07:30:00,1.619854e+12,2021-05-30 00:30:00,1.622335e+12,-14.728091,-0.048229,985.271909,999.951771


In [3]:
import sys
import rapidjson

path = "../freqtrade/mount/presets/test_strategy/config-backtesting.json"

with open(path) if path != '-' else sys.stdin as file:
    print(rapidjson.load(file))

{'max_open_trades': 1000, 'stake_currency': 'USDT', 'stake_amount': 15, 'tradable_balance_ratio': 0.99, 'fiat_display_currency': 'USD', 'dry_run': True, 'cancel_open_orders_on_exit': False, 'unfilledtimeout': {'buy': 10, 'sell': 30}, 'bid_strategy': {'ask_last_balance': 0.0, 'use_order_book': False, 'order_book_top': 1, 'check_depth_of_market': {'enabled': False, 'bids_to_ask_delta': 1}}, 'ask_strategy': {'use_order_book': False, 'order_book_min': 1, 'order_book_max': 1, 'use_sell_signal': True, 'sell_profit_only': False, 'ignore_roi_if_buy_signal': False}, 'exchange': {'name': 'binance', 'key': 'your_exchange_key', 'secret': 'your_exchange_secret', 'ccxt_config': {'enableRateLimit': True}, 'ccxt_async_config': {'enableRateLimit': True, 'rateLimit': 200}, 'pair_whitelist': ['BTC/USDT'], 'pair_blacklist': ['BNB/BTC']}, 'pairlists': [{'method': 'StaticPairList'}], 'edge': {'enabled': False, 'process_throttle_secs': 3600, 'calculate_since_number_of_days': 7, 'allowed_risk': 0.01, 'stoplos

In [10]:
from freqtrade.nbtools.helper import get_function_body, get_class_from_string
from freqtrade.nbtools.strategy import INbStrategy

strategy_code = get_function_body(strategy_func)
strategy = get_class_from_string(strategy_code, "NotebookStrategy")(self.config)

NameError: name 'self' is not defined

In [5]:
import os
if "freqtrade" not in os.listdir():
    os.chdir("..")
    import nest_asyncio
    nest_asyncio.apply()

from freqtrade.commands.optimize_commands import start_backtesting

# start_backtesting({
#     "datadir": "../freqtrade/mount/data/binance",
#     "userdir": ["../freqtrade/mount/presets/test_strategy"],
#     "config": ["../freqtrade/mount/presets/test_strategy/config-backtesting.json"],
#     "strategy_path": "../freqtrade/mount/presets/test_strategy/strategies",
#     "timeframe": "15m",
#     "timerange": "20210501-20210601",
#     "strategy": "NotebookStrategy",
# })

!python -m freqtrade backtesting --datadir ../freqtrade/mount/data/binance --userdir ../freqtrade/mount/presets/test_strategy --config ../freqtrade/mount/presets/test_strategy/config-backtesting.json --timeframe 15m --timerange 20210501-20210601 --strategy NotebookStrategy

Result for strategy NotebookStrategy
========================================================== BACKTESTING REPORT =========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
| BTC/USDT |    329 |          -0.28 |         -92.92 |           -13.952 |          -1.40 |        0:51:00 |   117     0   212  35.6 |
|    TOTAL |    329 |          -0.28 |         -92.92 |           -13.952 |          -1.40 |        0:51:00 |   117     0   212  35.6 |
===================================================== SELL REASON STATS =====================================================
|   Sell Reason |   Sells |   Win  Draws  Loss  Win% |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |
|---------------+---------+----

In [5]:
from freqtrade.nbtools.helper import get_readable_date
get_readable_date()

'2021-07-08_13:54:08'

In [6]:
from pathlib import Path
import os
import wandb
import pickle

def add_single_asset(path, project, asset_name):
    with wandb.init(project=project) as run:
        artifact = wandb.Artifact(asset_name, type="single")
        artifact.add_file(path, name=asset_name)
        run.log_artifact(artifact)

def retrieve_pickle_object(project, asset_name):
    with wandb.init(project=project) as run:
        artifact = run.use_artifact(f'{asset_name}:latest')
        filepath = (Path.cwd() / artifact.download()).glob("*")[0]
        with filepath.open("rb") as f:
            return pickle.load(f)
        
retrieve_pickle_object("test-artifacts", "model.pkl")

TypeError: 'generator' object is not subscriptable